In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge

from tqdm import tqdm

In [ ]:
def get_dataset(X, error=0.2):
    sigma = np.sqrt(error)
    Y = np.sin(X) + np.random.randn(X.shape[0])*sigma

    # poly features
    pf = PolynomialFeatures(degree=8, include_bias=False)
    X_poly = pf.fit_transform(X.reshape(-1, 1))

    return X_poly, Y

In [ ]:
def get_variance(predictions):
    return np.mean(np.var(predictions, axis=0))

def get_bias_squared(Xs, predictions):
    return np.mean((np.mean(predictions, axis=0) - np.sin(np.array(Xs)))**2)

In [ ]:
    n_train = 30
    n_test = 200
    X_train = np.random.uniform(-np.pi, np.pi, n_train)
    X_test = np.random.uniform(-np.pi, np.pi, n_test)
    trials = 100
    
    results = [] # (alpha, variance, bias^2)
    for alpha in tqdm(np.logspace(-10, 5, 300)):
        predictions = [] # contiene tutte le predizioni
        Xs = []
        for i in range(trials):
            # nuovo dataset
            X_poly_train, Y_train = get_dataset(X_train)

            ridge = Ridge(alpha=alpha, normalize=True)
            ridge.fit(X_poly_train, Y_train)
            X_poly_test, Y_test = get_dataset(X_test)
            y_hat = ridge.predict(X_poly_test)
            predictions.append(y_hat)
            print(Y_test)
            print(y_hat)
            mse = np.mean((Y_test - np.array(y_hat))**2)
        
        predictions = np.vstack(predictions)

        results.append((alpha, get_variance(predictions), get_bias_squared(X_test, predictions), mse))

In [ ]:
alphas, variances, bias_squared, mses = zip(*results)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
ax.scatter(np.log(alphas), variances, label='variance', s=3)
ax.scatter(np.log(alphas), bias_squared, label='bias_squared', s=3)
ax.scatter(np.log(alphas), mses, label='mse', s=3)
# ax.set_yscale('log')
plt.grid()
ax.legend()